HEATMAP:


In [ ]:
# יצירת Heatmap לערכים חסרים
plt.figure(figsize=(12, 8))
sns.heatmap(data_df.isnull(), cbar=False, cmap="viridis", yticklabels=False)
plt.title("Heatmap of Missing Values")
plt.xlabel("Variables")
plt.ylabel("Rows")
plt.show()


זיהוי ערכי קיצון באמצעות IQR ו-Z-Score:


In [ ]:

import numpy as np
from scipy.stats import zscore


# פונקציה לזיהוי ערכי קיצון
def detect_outliers(data, column):
    # חישוב IQR
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
   
    # זיהוי ערכי קיצון באמצעות Z-Score
    z_scores = zscore(data[column].dropna())
    z_outliers = (np.abs(z_scores) > 3)
   
    # שילוב שיטות IQR ו-Z-Score
    iqr_outliers = ((data[column] < lower_bound) | (data[column] > upper_bound))
    combined_outliers = iqr_outliers | z_outliers
   
    return combined_outliers


# הפעלת הפונקציה על עמודות מספריות
numeric_columns = data_df.select_dtypes(include=['float64', 'int64']).columns
outliers_summary = {col: detect_outliers(data_df, col).sum() for col in numeric_columns}


# יצירת סיכום ערכי קיצון
outliers_summary_df = pd.DataFrame(list(outliers_summary.items()), columns=['Variable', 'Outliers_Count'])
outliers_summary_df = outliers_summary_df[outliers_summary_df['Outliers_Count'] > 0]


# הצגת הסיכום
print(outliers_summary_df)




חישוב אחוזי הערכים החסרים:


In [ ]:

# חישוב אחוזי הערכים החסרים לכל משתנה
missing_percentage = data_df.isnull().mean() * 100


# יצירת DataFrame עם אחוזי החסרים
missing_summary = pd.DataFrame({
    'Variable': missing_percentage.index,
    'Missing_Percentage': missing_percentage.values
}).sort_values(by='Missing_Percentage', ascending=False)


# הצגת הסיכום
print(missing_summary)


בדיקת הקשר בין alb24h NA ל-dead_5y:


In [ ]:
# יצירת משתנה בינארי לחסרים
data_df['alb24h_missing'] = data_df['alb24h'].isnull().astype(int)


# טבלת שכיחות (Contingency Table)
alb24h_dead_contingency = pd.crosstab(data_df['alb24h_missing'], data_df['dead_5y'])


# מבחן חי-בריבוע (Chi-Square Test)
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(alb24h_dead_contingency)


# תוצאות
print("Chi-Square Test Results:")
print(f"Chi2 Statistic: {chi2}, p-value: {p}, Degrees of Freedom: {dof}")


מסקנה:
התוצאה של מבחן ה-Chi-Square מראה ש:
p-value = 0.449 > 0.05: אין קשר מובהק סטטיסטית בין חסר במשתנה alb24h (alb24h_missing) לבין משתנה המטרה dead_5y.
Chi2 Statistic נמוך: החסר בנתון זה לא משפיע באופן מובהק על משתנה המטרה.
הערכים החסרים במשתנה alb24h לא מקושרים סטטיסטית לתמותה תוך 5 שנים (dead_5y) ולכן כדאי למחוק את הנתון.

In [ ]:
# מחיקת המשתנה alb24h מהנתונים
data_df = data_df.drop(columns=['alb24h'])


# הצגת הנתונים לאחר המחיקה
data_df.info()


ביצוע אימפוטציה לשאר הערכים החסרים:


In [ ]:

from sklearn.impute import SimpleImputer


# משתנים קטגוריאליים: מילוי בערך השכיח (mode)
categorical_columns = ['gravity_u', 'nitrites_u', 'leuko_u', 'proteinuria']
imputer_mode = SimpleImputer(strategy='most_frequent')
data_df[categorical_columns] = imputer_mode.fit_transform(data_df[categorical_columns])


# משתנה רציף: מילוי בערך החציון
continuous_columns = ['albumin' , 'ACR']
imputer_median = SimpleImputer(strategy='median')
data_df[continuous_columns] = imputer_median.fit_transform(data_df[continuous_columns])


# בדיקת השלמת החסרים
missing_after_imputation = data_df.isnull().sum()


# הצגת מצב הערכים החסרים לאחר אימפוטציה
print(missing_after_imputation)

ערכים חריגים:


In [ ]:

# פונקציה לזיהוי ערכי קיצון
def detect_outliers(data, column):
    # חישוב IQR
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    # זיהוי ערכי קיצון באמצעות Z-Score
    z_scores = zscore(data[column].dropna())
    z_outliers = (np.abs(z_scores) > 3)


    # שילוב שיטות IQR ו-Z-Score
    iqr_outliers = ((data[column] < lower_bound) | (data[column] > upper_bound))
    combined_outliers = iqr_outliers | z_outliers


    return combined_outliers


# הפעלת הפונקציה על עמודות מספריות
numeric_columns = data_df.select_dtypes(include=['float64', 'int64']).columns
outliers_summary = {col: detect_outliers(data_df, col).sum() for col in numeric_columns}


# יצירת סיכום ערכי קיצון
outliers_summary_df = pd.DataFrame(list(outliers_summary.items()), columns=['Variable', 'Outliers_Count'])
outliers_summary_df = outliers_summary_df[outliers_summary_df['Outliers_Count'] > 0]


# מיון המשתנים לפי מספר ערכי הקיצון
outliers_summary_df_sorted = outliers_summary_df.sort_values(by='Outliers_Count', ascending=False)


# הצגת המשתנים עם הכי הרבה ערכי קיצון
print("משתנים עם הכי הרבה ערכי קיצון:")
print(outliers_summary_df_sorted.head())


בדיקה פרטנית של כל משתנה עם ערכי קיצון:


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro, mannwhitneyu


# פונקציה לניתוח משתנה עם ערכי קיצון
def analyze_outliers(data, column, target):
    try:
        # חישוב IQR
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR


        # סינון ערכי קיצון
        outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]


        # הדפסת סיכום סטטיסטי
        print(f"Analysis for {column}:")
        print(f"Total Outliers: {len(outliers)}")
        print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")


        # התפלגות המשתנה
        plt.figure(figsize=(8, 5))
        sns.histplot(data[column], kde=True, bins=30)
        plt.axvline(lower_bound, color='red', linestyle='--', label='Lower Bound')
        plt.axvline(upper_bound, color='red', linestyle='--', label='Upper Bound')
        plt.title(f'Distribution of {column}')
        plt.legend()
        plt.show()


        # בדיקה סטטיסטית של הקשר עם משתנה המטרה
        print(f"Testing {column} with {target}:")
        with_outliers = data[target][(data[column] >= lower_bound) & (data[column] <= upper_bound)]
        without_outliers = data[target][~((data[column] >= lower_bound) & (data[column] <= upper_bound))]


        # בדיקת נורמליות
        if len(with_outliers) > 3 and len(without_outliers) > 3:  # לוודא שיש מספיק נתונים
            shapiro_with = shapiro(with_outliers)
            shapiro_without = shapiro(without_outliers)


            if shapiro_with.pvalue > 0.05 and shapiro_without.pvalue > 0.05:
                # אם ההתפלגות נורמלית
                test_result = mannwhitneyu(with_outliers, without_outliers)
                print(f"Mann-Whitney Test: U-statistic={test_result.statistic}, p-value={test_result.pvalue}")
            else:
                print("The data is not normal. Non-parametric tests applied.")
        else:
            print("Not enough data for statistical test.")
    except Exception as e:
        print(f"Error analyzing {column}: {e}")


# משתנים עם ערכי קיצון לבדיקה
columns_with_outliers = ['immigrant', 'leuko_u', 'antidiabetics', 'ACR', 'CVD']


# בדיקה משתנה אחר משתנה
for column in columns_with_outliers:
    print(f"Analyzing {column}...")
    analyze_outliers(data_df, column, 'dead_5y')


הקשר בין ערכי ACR קיצוניים לDead_5y:


In [ ]:
from scipy.stats import mannwhitneyu


# חישוב גבולות ערכי הקיצון עבור ACR
Q1 = data_df['ACR'].quantile(0.25)
Q3 = data_df['ACR'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# פילוח הנתונים עם ובלי ערכי קיצון
data_within_bounds = data_df[(data_df['ACR'] >= lower_bound) & (data_df['ACR'] <= upper_bound)]
data_outliers = data_df[(data_df['ACR'] < lower_bound) | (data_df['ACR'] > upper_bound)]


# חישוב אחוז התמותה (dead_5y) בקבוצות
death_rate_within = data_within_bounds['dead_5y'].mean()
death_rate_outliers = data_outliers['dead_5y'].mean()


print(f"Death Rate within bounds: {death_rate_within:.2%}")
print(f"Death Rate in outliers: {death_rate_outliers:.2%}")


# מבחן סטטיסטי להשוואת התפלגויות (Mann-Whitney U Test)
stat, p_value = mannwhitneyu(data_within_bounds['ACR'], data_outliers['ACR'])
print(f"Mann-Whitney U Test: U-statistic={stat}, p-value={p_value}")


# יצירת גרף השוואה בין ערכי הקיצון ומשתנה המטרה
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(8, 6))
sns.boxplot(data=data_df, x='dead_5y', y='ACR', showfliers=True)
plt.title("Boxplot of ACR by Dead_5y")
plt.xlabel("Dead_5y")
plt.ylabel("ACR")
plt.show()


בדיקת מובהקות ACR וDead_5y:


In [ ]:

# פילוח נתוני ACR לערכים בטווח ולערכי קיצון
Q1 = data_df['ACR'].quantile(0.25)
Q3 = data_df['ACR'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


acr_within_bounds = data_df[(data_df['ACR'] >= lower_bound) & (data_df['ACR'] <= upper_bound)]['ACR']
acr_outliers = data_df[(data_df['ACR'] < lower_bound) | (data_df['ACR'] > upper_bound)]['ACR']


# בדיקה אם שתי הקבוצות אינן ריקות
if len(acr_within_bounds) > 0 and len(acr_outliers) > 0:
    # ביצוע מבחן Mann-Whitney U
    stat, p_value = mannwhitneyu(acr_within_bounds, acr_outliers, alternative='two-sided')
    print(f"Mann-Whitney U Test Results:")
    print(f"U-statistic: {stat}, p-value: {p_value}")
else:
    print("One or both groups are empty, cannot perform Mann-Whitney U test.")


השפעת ערכי הקיצון על הקורלציה וההתפלגות:


In [ ]:

# שימוש בניתוח לוגריתמי או קטגוריאלי
data_df['ACR_transformed'] = np.log1p(data_df['ACR'])
#ניתוח מחדש עם הנתונים המעובדים:
corr_transformed, p_transformed = spearmanr(data_df['ACR_transformed'], data_df['dead_5y'])
print(f"Correlation with transformed data: {corr_transformed}, p-value: {p_transformed}")


הקשר בין ערכי CVD קיצוניים לDead_5y:


In [ ]:

# חישוב גבולות ערכי הקיצון עבור CVD
Q1 = data_df['CVD'].quantile(0.25)
Q3 = data_df['CVD'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# פילוח הנתונים עם ובלי ערכי קיצון
data_within_bounds = data_df[(data_df['CVD'] >= lower_bound) & (data_df['CVD'] <= upper_bound)]
data_outliers = data_df[(data_df['CVD'] < lower_bound) | (data_df['CVD'] > upper_bound)]


# חישוב אחוז התמותה (dead_5y) בקבוצות
death_rate_within = data_within_bounds['dead_5y'].mean()
death_rate_outliers = data_outliers['dead_5y'].mean()


print(f"Death Rate within bounds: {death_rate_within:.2%}")
print(f"Death Rate in outliers: {death_rate_outliers:.2%}")


# מבחן סטטיסטי להשוואת התפלגויות (Mann-Whitney U Test)
stat, p_value = mannwhitneyu(data_within_bounds['CVD'], data_outliers['CVD'])
print(f"Mann-Whitney U Test: U-statistic={stat}, p-value={p_value}")


# יצירת גרף השוואה בין ערכי הקיצון ומשתנה המטרה
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(8, 6))
sns.boxplot(data=data_df, x='dead_5y', y='CVD', showfliers=True)
plt.title("Boxplot of CVD by Dead_5y")
plt.xlabel("Dead_5y")
plt.ylabel("CVD")
plt.show()


בדיקת השפעת ערכי קיצון על קורלציה ושונות התפלגות במשתנה CVD:


In [ ]:
from scipy.stats import spearmanr, ks_2samp


# חישוב קורלציה עם ערכי הקיצון
corr_with_outliers, p_with_outliers = spearmanr(data_df['CVD'], data_df['dead_5y'])


# הסרת ערכי הקיצון
Q1 = data_df['CVD'].quantile(0.25)
Q3 = data_df['CVD'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data_without_outliers = data_df[(data_df['CVD'] >= lower_bound) & (data_df['CVD'] <= upper_bound)]


# חישוב קורלציה ללא ערכי הקיצון
corr_without_outliers, p_without_outliers = spearmanr(data_without_outliers['CVD'], data_without_outliers['dead_5y'])


# בדיקת שוני בהתפלגות (Kolmogorov-Smirnov)
ks_stat, ks_p_value = ks_2samp(
    data_df['CVD'].dropna(),
    data_without_outliers['CVD'].dropna()
)


# תוצאות
print(f"Correlation with outliers: {corr_with_outliers}, p-value: {p_with_outliers}")
print(f"Correlation without outliers: {corr_without_outliers}, p-value: {p_without_outliers}")
print(f"Kolmogorov-Smirnov Test: KS Statistic={ks_stat}, p-value={ks_p_value}")


# מסקנות:
if p_with_outliers < 0.05 and p_without_outliers >= 0.05:
    print("הקורלציה מובהקת עם ערכי הקיצון, אך לא מובהקת בלעדיהם. מומלץ להסיר את ערכי הקיצון.")
elif p_with_outliers < 0.05 and p_without_outliers < 0.05 and ks_p_value < 0.05:
    print("הקורלציה נשארת מובהקת, אך יש שוני מובהק בהתפלגות. ייתכן שכדאי למחוק את ערכי הקיצון.")
elif p_with_outliers < 0.05 and p_without_outliers < 0.05 and ks_p_value >= 0.05:
    print("אין שוני מובהק בהתפלגות ואין שינוי משמעותי בקורלציה. אין צורך למחוק את ערכי הקיצון.")
else:
    print("לא נמצאה קורלציה מובהקת עם או בלי ערכי הקיצון. אין צורך למחוק.")


leuko_u:


In [ ]:
# חישוב גבולות ערכי הקיצון עבור leuko_u
Q1 = data_df['leuko_u'].quantile(0.25)
Q3 = data_df['leuko_u'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# פילוח הנתונים עם ובלי ערכי קיצון
data_with_outliers = data_df.copy()
data_without_outliers = data_df[(data_df['leuko_u'] >= lower_bound) & (data_df['leuko_u'] <= upper_bound)]


# בדיקת הקורלציה עם ובלי ערכי הקיצון
corr_with_outliers, p_with_outliers = spearmanr(data_with_outliers['leuko_u'], data_with_outliers['dead_5y'])
corr_without_outliers, p_without_outliers = spearmanr(data_without_outliers['leuko_u'], data_without_outliers['dead_5y'])


# מבחן Kolmogorov-Smirnov להשוואת התפלגויות
ks_stat, ks_p_value = ks_2samp(data_with_outliers['leuko_u'], data_without_outliers['leuko_u'])


# תוצאות
print(f"Correlation with outliers: {corr_with_outliers}, p-value: {p_with_outliers}")
print(f"Correlation without outliers: {corr_without_outliers}, p-value: {p_without_outliers}")
print(f"Kolmogorov-Smirnov Test: KS Statistic={ks_stat}, p-value={ks_p_value}")


# גרף ההתפלגות לפני ואחרי הסרת ערכי קיצון
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(10, 6))
sns.histplot(data_with_outliers['leuko_u'], kde=True, color='blue', label='With Outliers', bins=30)
sns.histplot(data_without_outliers['leuko_u'], kde=True, color='orange', label='Without Outliers', bins=30)
plt.axvline(lower_bound, color='red', linestyle='--', label='Lower Bound')
plt.axvline(upper_bound, color='red', linestyle='--', label='Upper Bound')
plt.title('Distribution of leuko_u with and without Outliers')
plt.legend()
plt.show()


כדי לבצע טרנספורמציה למשתנה leuko_u, ננסה את הטרנספורמציה הלוגריתמית (log1p) מכיוון שהיא מתאימה למשתנים עם ערכים חיוביים וזנבות ארוכים. 

In [ ]:


# יצירת משתנה עם טרנספורמציה לוגריתמית
data_df['leuko_u_transformed'] = np.log1p(data_df['leuko_u'])


# חישוב קורלציה עם ובלי ערכי קיצון לאחר הטרנספורמציה
corr_transformed, p_transformed = spearmanr(data_df['leuko_u_transformed'], data_df['dead_5y'])
print(f"Correlation with transformed data: {corr_transformed}, p-value: {p_transformed}")


# פילוח הנתונים עם ובלי ערכי קיצון לאחר הטרנספורמציה
Q1_transformed = data_df['leuko_u_transformed'].quantile(0.25)
Q3_transformed = data_df['leuko_u_transformed'].quantile(0.75)
IQR_transformed = Q3_transformed - Q1_transformed
lower_bound_transformed = Q1_transformed - 1.5 * IQR_transformed
upper_bound_transformed = Q3_transformed + 1.5 * IQR_transformed


data_within_bounds_transformed = data_df[
    (data_df['leuko_u_transformed'] >= lower_bound_transformed) &
    (data_df['leuko_u_transformed'] <= upper_bound_transformed)
]
data_outliers_transformed = data_df[
    (data_df['leuko_u_transformed'] < lower_bound_transformed) |
    (data_df['leuko_u_transformed'] > upper_bound_transformed)
]


# חישוב אחוז התמותה בקבוצות
death_rate_within_transformed = data_within_bounds_transformed['dead_5y'].mean()
death_rate_outliers_transformed = data_outliers_transformed['dead_5y'].mean()


print(f"Death Rate within bounds after transformation: {death_rate_within_transformed:.2%}")
print(f"Death Rate in outliers after transformation: {death_rate_outliers_transformed:.2%}")


# גרף השוואה בין ערכי הקיצון ומשתנה המטרה לאחר הטרנספורמציה
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(8, 6))
sns.boxplot(data=data_df, x='dead_5y', y='leuko_u_transformed', showfliers=True)
plt.title("Boxplot of Transformed Leuko_u by Dead_5y")
plt.xlabel("Dead_5y")
plt.ylabel("Leuko_u (Transformed)")
plt.show()


immigrant:


In [ ]:

# יצירת טבלת שכיחות (Contingency Table)
contingency_table = pd.crosstab(data_df['immigrant'], data_df['dead_5y'])


# ביצוע מבחן חי-בריבוע (Chi-Square Test)
chi2, p, dof, expected = chi2_contingency(contingency_table)


# הדפסת תוצאות
print("Chi-Square Test Results:")
print(f"Chi2 Statistic: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)


ביצוע טרנספורמציה לACR:


In [ ]:

# טרנספורמציה לוגריתמית על ACR (Log1p עבור טיפול בערכים שליליים או אפס)
data_df['ACR_log'] = np.log1p(data_df['ACR'])


# בדיקת ההתפלגות לאחר הטרנספורמציה
plt.figure(figsize=(8, 6))
sns.histplot(data_df['ACR_log'], kde=True, bins=30, color='blue')
plt.title('Distribution of Transformed ACR (Log1p)')
plt.xlabel('ACR_log')
plt.ylabel('Frequency')
plt.show()


# הצגת סטטיסטיקות תיאוריות למשתנה המותמר
print("\nStatistical summary for transformed ACR:\n", data_df['ACR_log'].describe())